## Original dataset

In [1]:
import datasets
import json

In [5]:
with open('v2/train.json') as f:
    data = json.load(f)

In [61]:
## print(f'{len(data)} pictures')
key0 = list(data.keys())[0] # -> Picture name
print(f"The chosen picture is '{key0}'")
print('')

val0 = data[key0]
print(f'{len(val0)} games involving this picture')
key00 = list(val0.keys())[0] # Game ID ranging from 0 to 2501
print(f"The chosen game is '{key00}'")
print('')

val00 = val0[key00]
print(f'data[{key0}][{key00}] is {type(val00)}')
print(f'{len(val00)} elements in data[{key0}][{key00}]')
print('')

val000 = val00[0]
print(f'{len(val000)} keys in data[{key0}][{key00}][0]')
print(list(val000))
print('')

for key in val000:
    print(f'{key} : {val000[key]}')

The chosen picture is 'person_truck/COCO_train2014_000000258505.jpg'

20 games involving this picture
The chosen game is '2389'

data[person_truck/COCO_train2014_000000258505.jpg][2389] is <class 'list'>
3 elements in data[person_truck/COCO_train2014_000000258505.jpg][2389]

38 keys in data[person_truck/COCO_train2014_000000258505.jpg][2389][0]
['Game_ID', 'Round_Nr', 'Message_Nr', 'Message_Speaker', 'Message_Type', 'Message_Text', 'Round_Common', 'Round_Images_A', 'Round_Images_B', 'Game_Domain_ID', 'Game_Domain_1', 'Game_Domain_2', 'Feedback_A', 'Feedback_B', 'Agent_1', 'Agent_2', 'Round_Highlighted_A', 'Round_Highlighted_B', 'Message_Timestamp', 'Message_Turn', 'Message_Agent_ID', 'Game Duration', 'N_Messages_In_Round', 'Round Duration', 'Total_Game_Score', 'Game_Scores', 'Round_Scores', 'Total_Round_Score', 'In_Segment', 'Reason', 'Tokens', 'score', 'Discriminative_Features', 'All_features', 'Precision_Score', 'Recall_Score', 'F1_Score', 'Meteor_Score']

Game_ID : 2389
Round_Nr : 1

In [73]:
for pic in data:
    for game in data[pic]:
        for msg in data[pic][game]:
#             if msg['Game_ID'] == 2389 and msg['Round_Nr'] == 1:
#                 print(msg['Message_Nr'], msg['Message_Speaker'])
            if msg['Game_ID'] == 2389 and 

6 A
11 B
0 B
4 A


## Log file with clean sections

In [2]:
from datasets import load_dataset
import pandas as pd
import pickle
import sys
from processor import Log

In [182]:
# list, len=3
test_clean = pd.read_pickle('../data/test_clean_sections.pickle')
tc_df = pd.DataFrame(test_clean)
tc_df = tc_df.drop(0, axis=1)
tc_df[1][0]

[{'agent_id': 782,
  'segments': [(980,
    'do you have a man sitting on a black motorcyle on a white line?'),
   (782,
    'No I do not see a white line, I have a man sitting on a motorcycle though'),
   (980, 'do you have a red motorcycle with a cup in front of it'),
   (782, 'Yes I do'),
   (980, 'do you have a man on a bike next to a red sports car'),
   (782, 'No I do not'),
   (980, 'do you have any questions'),
   (782, 'Oh wait does the red sports car have dogs in it?'),
   (980, 'no'),
   (782, 'Ok then no')],
  'image_set': ['car_motorcycle/COCO_train2014_000000477807.jpg',
   'car_motorcycle/COCO_train2014_000000024608.jpg',
   'car_motorcycle/COCO_train2014_000000514839.jpg',
   'car_motorcycle/COCO_train2014_000000293647.jpg',
   'car_motorcycle/COCO_train2014_000000480807.jpg',
   'car_motorcycle/COCO_train2014_000000144597.jpg'],
  'clip_scores': array([[0.5757, 0.662 , 0.754 , 0.5586, 0.5586, 0.627 ],
         [0.6465, 0.6694, 0.7466, 0.583 , 0.612 , 0.628 ],
         

In [95]:
curr = tc_df[1][0] # list, len=10
curr = curr[9] # dict, keys=['agent_id', 'segments', 'image_set', 'clip_scores']
curr['segments']

[(782, 'Do you have a man on a motorcycle next to a red sports car?'),
 (980, 'do you have a motorcycle with a man on it at the gas station?'),
 (980, 'yes I do'),
 (782, 'No I dont'),
 (980, 'do you have a red motorcycle with a cup in front of it'),
 (782, 'Yes I do.'),
 (980, 'I have no more questions do you?'),
 (782,
  'Do you have a man on a motorcycle that has its front tire on a white line?'),
 (980, 'no i do not')]

## Image Features

In [25]:
img_feats = pd.read_pickle('data/image_feats.pickle')

In [33]:
key0 = list(img_feats.keys())[0]
img_feats[key0].shape

torch.Size([512, 16, 16])

# Log files

In [46]:
import os
import json

In [41]:
def load_logs(log_repository, data_path):

    filepath = os.path.join(data_path, log_repository)
    print("Loading logs from {}...".format(filepath))

    missing_counter = 0
    file_count = 0
    for _, _, files in os.walk(filepath):
        file_count += len(files)
    print("{} files found.".format(file_count))
    logs = []
    for root, dirs, files in os.walk(filepath):
        for file in files:
            if file.endswith(".json"):
                with open(os.path.join(root, file), 'r') as logfile:
                    log = Log(json.load(logfile))
                    if log.complete:
                        logs.append(log)

    print("DONE. Loaded {} completed game logs.".format(len(logs)))
    return logs

In [47]:
data_path = ".."
logs = load_logs("logs", data_path)

Loading logs from ../logs...
2502 files found.
DONE. Loaded 2502 completed game logs.


In [309]:
def print_round(r):

    def print_images(r, player):
        print(f'\t{player}:')
        for i, image in enumerate(r.images[player]):
            common = ' (Common)' if image in r.common else '         '
            marked = ' (Marked)' if r.highlighted[player][i] else ''
            print('\t\t' + image + common + marked)
    
    print("total score: {}".format(r.total_score))        
    print("player scores: A - {}, B - {}".format(r.scores["A"], r.scores["B"]))

    print('images:')
    print_images(r, 'A')
    print_images(r, 'B')
    
    print('\nmessages:')
    msg_list = []
    for i, m in enumerate(r.messages):
        if m.type == "text":
            msg_list.append({'speaker': m.speaker, 'text': m.text})
        if m.type == "selection":
            speaker = m.speaker
            img = m.text.split()[2]
            label = "common" if m.text.split()[1] == "" else "different"
            correct = 'correctly' if img in r.common else 'incorrectly'
            if msg_list != []:
                msg_list[-1]['text'] += f' ({speaker} {correct} marked {img} as {label})'
    for d in msg_list:
        speaker = d['speaker']
        text = d['text']
        print(f'\t{speaker}: {text}')
        
thisGame = logs[0] # Log, containing 5 rounds
thisRound = logs[0].rounds[0]
print_round(thisRound)

total score: 6
player scores: A - 3, B - 3
images:
	A:
		person_truck/COCO_train2014_000000418229.jpg (Common) (Marked)
		person_truck/COCO_train2014_000000227003.jpg          (Marked)
		person_truck/COCO_train2014_000000265781.jpg (Common)
		person_truck/COCO_train2014_000000258505.jpg (Common) (Marked)
		person_truck/COCO_train2014_000000032674.jpg         
		person_truck/COCO_train2014_000000190016.jpg (Common)
	B:
		person_truck/COCO_train2014_000000185945.jpg         
		person_truck/COCO_train2014_000000265781.jpg (Common)
		person_truck/COCO_train2014_000000237372.jpg          (Marked)
		person_truck/COCO_train2014_000000418229.jpg (Common) (Marked)
		person_truck/COCO_train2014_000000190016.jpg (Common)
		person_truck/COCO_train2014_000000258505.jpg (Common) (Marked)

messages:
	A: Do you have 2 men, 1 holding a stop sign?
	B: no fo you have what looks to be a road being paved
	B: or sprayed with a man standing behind it (B correctly marked person_truck/COCO_train2014_0000004182

In [185]:
from transformers import DebertaTokenizer

In [306]:
def round2dict(r, tokenizer, player):
    input_ids = []
    token_type_ids = []
    labels = []
    
    images = r.images[player]
    # 0 = undecided, 1 = common, 2 = different
    image_status = [[0] for _ in images]
    
    for i, m in enumerate(r.messages):
        if m.type == "text":
            tokenized_msg =  tokenizer(m.text,padding=False,truncation=True)['input_ids']
            input_ids.append(tokenized_msg)
            token_type_id = 0 if m.speaker == player else 1
            token_type_ids.append([token_type_id for _ in tokenized_msg])
            labels.append([x * len(tokenized_msg) for x in image_status])

        if m.type == "selection" and m.speaker == player:
            img = m.text.split()[2]
            img_index = images.index(img)
            label = 1 if m.text.split()[1] == "" else 2
            image_status[img_index] = [label]

    return {'input_ids':input_ids, 'token_type_ids':token_type_ids, 'labels':labels}

In [307]:
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")

In [308]:
thisGame = logs[0] # Log, containing 5 rounds
thisRound = logs[0].rounds[0]

result = round2dict(thisRound, tokenizer, 'A')

result

{'input_ids': [[1,
   8275,
   47,
   33,
   132,
   604,
   6,
   112,
   1826,
   10,
   912,
   1203,
   116,
   2],
  [1, 2362, 9565, 47, 33, 99, 1326, 7, 28, 10, 921, 145, 21100, 2],
  [1, 368, 26998, 19, 10, 313, 2934, 639, 24, 2],
  [1, 10932, 6, 1437, 38, 109, 4, 1832, 47, 33, 1159, 816, 4191, 2],
  [1, 10932, 11, 2272, 8, 5718, 17753, 2],
  [1, 10932, 2],
  [1,
   8275,
   47,
   33,
   132,
   604,
   2934,
   11,
   760,
   9,
   10,
   2484,
   6,
   112,
   1826,
   10,
   912,
   1203,
   116,
   2],
  [1,
   2362,
   53,
   109,
   1717,
   33,
   10,
   693,
   19,
   10,
   909,
   16676,
   11,
   760,
   9,
   10,
   793,
   8517,
   2],
  [1, 3084, 38, 109, 45, 33, 14, 65, 2],
  [1, 11913, 97, 1142, 116, 2],
  [1, 2362, 939, 300, 127, 155, 2446, 2],
  [1, 13987, 47, 6, 162, 350, 2]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,